In [2]:
import numpy, matplotlib, time
import line_profiler
%load_ext line_profiler
directory = '../data/'
file_paths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

In [3]:
class Library():
    def __init__(self,index,N,T,M, book_ids: set):
        self.id = index
        self.size  = N
        self.signup_time = T
        self.books_per_day = M
        self.book_ids = book_ids
    
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
        
        

def process_file(filePath):
    with open(directory+filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    bookValues = tuple([int(n) for n in content[1].split()])
    libraries = []
    for i in range(L):
        N,T,M = content[2+2*i].split()
        N,T,M=int(N),int(T),int(M)
        book_ids = set([int(id) for id in content[2+2*i+1].split()])
        libraries.append(Library(i,N,T,M,book_ids))
    return ((B,L,D), bookValues, libraries)

In [4]:
def check_solution(D, libraries):
#     print("D ",D)
    days = 0
    prev_books = set()
    for library in libraries:
#         print("#################")
#         print("Library id", library.id)
#         print("signup", library.signup_time)
#         print("Books per day", library.books_per_day)
#         print("all books", len(library.book_ids))
# #         print("##############")
        days+=library.signup_time
#         if len(library.book_ids) > (D-days)*library.books_per_day:
#             print("what", len(library.book_ids), (D-days)*library.books_per_day)
#         elif len(library.book_ids) < (D-days)*library.books_per_day:
#             print("warning", len(library.book_ids), (D-days)*library.books_per_day)
        assert(not any([(book in prev_books) for book in library.book_ids]))
        prev_books.update(library.book_ids)
    assert(days <= D)

In [5]:
def get_score(file_path):
    (B,L,D),bookValues, libraries = process_file(file_path)
#     print("B L D",B,L,D)
    chosen_libraries = []
    already_scanned_books = set()
    day = 0
    def book_sort(book):
        return bookValues[book]
    def library_score(lib):
#         print(D,day,lib.signup_time)
        time = D - day - lib.signup_time
        return sum(sorted(lib.book_ids,key = book_sort, reverse=True)[:min(time*lib.books_per_day,lib.size)])/lib.signup_time
    
    while libraries:
        library_scores = [library_score(lib) for lib in libraries]
        next_library_index = library_scores.index(max(library_scores))
        next_library = libraries[next_library_index]
        day+=next_library.signup_time
        if day >= D:
            break
        next_library.book_ids = set(sorted(next_library.book_ids, key = book_sort, reverse=True)[:min((D-day)*next_library.books_per_day,len(next_library.book_ids))])
        already_scanned_books.update(next_library.book_ids)
        chosen_libraries.append(next_library)
        libraries.pop(next_library_index)
        for library in libraries:
            library.book_ids-=next_library.book_ids
        
#     print(len(chosen_libraries))
    score = 0
    for library in chosen_libraries:
        for idk in library.book_ids:
            score+=bookValues[idk]
    check_solution(D,chosen_libraries)
    return score
sum_score = 0
for file_path in file_paths:
    print(file_path)
    %time score=get_score(file_path)
    sum_score += score
    print(score)
print("SUM SCORE", sum_score)

a_example.txt
CPU times: user 0 ns, sys: 1.81 ms, total: 1.81 ms
Wall time: 1.09 ms
21
b_read_on.txt
CPU times: user 600 ms, sys: 7.47 ms, total: 607 ms
Wall time: 605 ms
5278400
c_incunabula.txt
CPU times: user 56.8 s, sys: 3.6 ms, total: 56.8 s
Wall time: 56.8 s
5571679
d_tough_choices.txt
CPU times: user 13min 38s, sys: 75.3 ms, total: 13min 38s
Wall time: 13min 39s
4976985
e_so_many_books.txt
CPU times: user 18.8 s, sys: 37.9 ms, total: 18.9 s
Wall time: 18.9 s
4682967
f_libraries_of_the_world.txt
CPU times: user 3.43 s, sys: 8 ms, total: 3.44 s
Wall time: 3.44 s
5224326
SUM SCORE 25734378


In [5]:
# LIBRAIES = []
# BOOK_SCORES = []

# #individual [list of ids of chosen libraries in order]
# def GE():
#     startTime = time.time()
#     population = []#ids
#     scores = []
#     while time.time() < startTime + 290:
        
    

In [6]:
# def get_score(chosen_libraries, days):
#     for library in chosen_libraries:
        

In [9]:
def get_score(file_path):
    (B,L,D),bookValues, libraries = process_file(file_path)
#     print("B L D",B,L,D)
    already_scanned_books = set()
    day = 0
    def book_sort(book):
        return bookValues[book]
    def library_score(lib):
        time = D - lib.signup_time
        return sum(sorted(lib.book_ids,key = book_sort, reverse=True)[:min(time*lib.books_per_day,lib.size)])/lib.signup_time
#     print(time.time())
    sorted_libraries = sorted(libraries,key=library_score, reverse=True)
#     print(time.time())
    it=0;
    while it < len(sorted_libraries):
        next_library = sorted_libraries[it]
        day+=next_library.signup_time
        if day>=D:
            break
        next_library.book_ids = next_library.book_ids - already_scanned_books
#         print("Max capacity", (D-day)*next_library.books_per_day)
        next_library.book_ids = set(sorted(next_library.book_ids, key = book_sort, reverse=True)[:min((D-day)*next_library.books_per_day,len(next_library.book_ids))])
#         print("Actual capacity", len(next_library.book_ids))
        already_scanned_books.update(next_library.book_ids)
        sorted_libraries[it] = next_library
        it+=1
#     print(time.time())
    score = 0
    for i in range(0,it):
        for idk in sorted_libraries[i].book_ids:
            score+=bookValues[idk]
#     print("SCORE ",score)
    
    check_solution(D, sorted_libraries[:it])
    return score
sum_scores = 0
for file_path in file_paths:
    print(file_path)
    %timeit -n 10 get_score(file_path)
print("SUM SCORES", sum_scores)

a_example.txt
68.8 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
b_read_on.txt
78.6 ms ± 412 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
c_incunabula.txt
161 ms ± 600 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
d_tough_choices.txt
334 ms ± 2.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
e_so_many_books.txt
318 ms ± 467 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
f_libraries_of_the_world.txt
316 ms ± 961 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
SUM SCORES 0
